In [8]:
import random
import math
import copy
import numpy as np
from time import time



def read_bpp_file(file_path):
    with open(file_path, 'r') as f:
        n = int(f.readline().strip())
        capacity = int(f.readline().strip())
        objects = [int(line.strip()) for line in f]
    return (n, capacity, objects)



#(n , bin_capacity , items ) = read_bpp_file('Hard28_BPP900.txt')  # optimal 75
#(n , bin_capacity , items ) = read_bpp_file('Hard28_BPP709.txt')  # optimal 67
#(n , bin_capacity , items ) = read_bpp_file('BPP_100_100_0.2_0.7_8.txt')  # optimal 46
(n , bin_capacity , items ) = read_bpp_file('BPP_50_200_0.1_0.8_4.txt')  # optimal 25
print(f"Nombre d'objets: {n}, capacité: {bin_capacity}\n")



# Test the algorithm
nb_iter_FIFA = 50
nb_continents = 5
nb_teams = 4
num_whales = nb_continents * nb_teams
num_items = n


class Item:
    def __init__(self, label: int = None, size: int = None):
        self._label = label  # index
        self._size = size  # weight


class Bin:
    def __init__(self, items: list = None, capacity: int = 0):
        if items is None:
            items = []
        self._items = items
        self._capacity = capacity

    def add_item(self, item: Item):
        self._items.append(item)
        self._capacity += item._size

    def get_labels(self) -> list[int]:  # get indexes
        return [item._label for item in self._items]


class Solution:
    def __init__(self, bins: list[Bin] = None, max_capacity: int = bin_capacity):
        if bins is None:
            bins = []
        self._bins = bins
        self._max_capacity = max_capacity
        self._fitness = self.calculate_fitness()

    def calculate_fitness(self):
        """somme(taille bin / max)² for each bin / nb bins"""
        '''if self._bins is []:
            return 1
        num_bins = len(self._bins)
        bins_capacity = np.asarray([_bin._capacity for _bin in self._bins], dtype=int)
        self._fitness = np.sum(np.power(bins_capacity / self._max_capacity, 2)) / num_bins
        return self._fitness'''
        self._fitness = len(self._bins) + sum((bin_capacity - bin._capacity / bin_capacity) for bin in self._bins) / (bin_capacity * len(self._bins))
        return self._fitness

    def count_items(self):
        return len([item for _bin in self._bins for item in _bin._items])

    def count_indexes(self):
        items = [item for _bin in self._bins for item in _bin._items]
        indexes = [item._label for item in items]
        return len(set(indexes))

    def affichage(self):
        return [bin.get_labels() for bin in self._bins]

    def __str__(self):
        return f"{self._fitness}: {affichage(self.affichage())}"



def next_fit(indexes: list[int]) -> Solution:
    random.shuffle(indexes)
    bins = []
    current_bin = Bin()
    for i in indexes:
        if current_bin._capacity + items[i] <= bin_capacity:
            current_bin.add_item(Item(label=i, size=items[i]))
        else:
            bins.append(current_bin)
            current_bin = Bin()
            current_bin.add_item(Item(label=i, size=items[i]))
    bins.append(current_bin)
    return Solution(bins)


def affichage(list_index: list[int]) -> list[int]:
    return [[items[i] for i in bin] for bin in list_index]


"""def generate_continent(n_teams: int) -> list[Solution]:
    indexes = list(items.keys())
    return np.array([next_fit(indexes) for _ in range(n_teams)], dtype=object)"""


def generate_preliminary(n_contients: int, n_teams: int) -> list[list[Solution]]:
    global items


    population = initialize_population(num_whales, items, bin_capacity)
    solb, pop = opt(population)

    items = {i:items[i] for i in range(n)}
    continents = []
    continent = []
    for i, sol in enumerate(pop):
        team = []
        for bin in sol:
            current_bin = Bin()
            for item in bin:
                current_bin.add_item(Item(label=item, size=items[item]))
            assert current_bin._capacity <= bin_capacity
            team.append(current_bin)
        if i != 0 and i % n_teams == 0:
            continents.append(continent)
        else:
            continent.append(Solution(team))
    return np.array(continents, dtype=object)


def crossover(father: Solution, mother: Solution):
    while True:
        two_points_father = np.random.randint(len(father._bins) + 1, size=2)
        two_points_mother = np.random.randint(len(mother._bins) + 1, size=2)
        if len(set(two_points_father)) == 2 and len(set(two_points_mother)) == 2:
            break
    two_points_father.sort()
    two_points_mother.sort()
    """choisir deux indexes for father and mother to do croisement a 2 points"""
    first_child = crossover_one_chromosome(chromosome=father, points_insert=two_points_father[1],
                                                genes=mother._bins[two_points_mother[0]:two_points_mother[1]])
    second_child = crossover_one_chromosome(chromosome=mother, points_insert=two_points_mother[0],
                                                genes=father._bins[two_points_father[0]:two_points_father[1]])
    return first_child, second_child


def crossover_one_chromosome(chromosome: Solution, points_insert: int, genes: list):
    """insert genes then completes with genes from chromosome"""
    child = Solution(genes, max_capacity=chromosome._max_capacity)
    take_labels = [_bin.get_labels() for _bin in genes]
    take_labels = [label for bin_labels in take_labels for label in bin_labels]
    left_bins = []
    right_bins = []
    free_items = []
    for i in range(len(chromosome._bins)):
        intersection = [label for label in chromosome._bins[i].get_labels() if label in take_labels]
        if len(intersection) == 0:
            if i < points_insert:
                left_bins.append(chromosome._bins[i])
            else:
                right_bins.append(chromosome._bins[i])
        else:
            for item in chromosome._bins[i]._items:
                if item._label not in intersection:
                    free_items.append(item)
    child._bins = left_bins + child._bins + right_bins
    return first_fit_descending(child, free_items)


def first_fit_descending(chromosome: Solution, items: list):
    """try to add leftover items in child's current bins else add new ones"""
    # chromosome = copy.deepcopy(chromosome)
    objects = copy.deepcopy(items)
    objects = sorted(objects, key=lambda item: item._size, reverse=True)
    max_capacity = chromosome._max_capacity
    add_bins = []
    current_bin = Bin()
    _flag = False
    _count = 0
    for item in objects:
        _count += 1
        for _bin in chromosome._bins:
            if _bin._capacity + item._size <= max_capacity:
                _bin.add_item(item)
                _flag = True
                break
        if not _flag:
            if current_bin._capacity + item._size <= max_capacity:
                current_bin.add_item(item)
            else:
                add_bins.append(current_bin)
                current_bin = Bin()
                current_bin.add_item(item)
        _flag = False
    if current_bin._capacity > 0:
        add_bins.append(current_bin)
    chromosome._bins += add_bins
    chromosome.calculate_fitness()
    return chromosome


def FIFA(n_iter: int, n_continents: int, n_teams: int):
    best_sol = None
    best_score = n
    start_time = time()

    preliminary = generate_preliminary(n_continents, n_teams)  # list of continents

    for p in range(n_iter):
        finals = []
        for continent in preliminary:
            team1, team2 = sorted(continent, reverse=False, key=lambda team: team._fitness)[:2]
            finals.extend([team1, team2])

        champion = sorted(finals, reverse=False, key=lambda team: team._fitness)[0]
        if champion._fitness < best_score:
            # print('found new best', champion._fitness, best_score, len(champion._bins))
            best_sol = champion
            best_score = champion._fitness

        next_preliminary = []
        print
        if p < n_iter - 1:
            for i in range(0, len(finals), 2):
                continent = finals[i: i + 2]
                for _ in range(0, n_teams - 2, 2):
                    continent.extend(crossover(continent[0], continent[1]))
                next_preliminary.append(continent)
            preliminary = next_preliminary
    # best_sol._bins.sort(reverse=True, key=lambda bin: bin._capacity if bin._capacity > bin_capacity // 2 else min(item._size for item in bin._items))
    fifi_time = time() - start_time
    return best_sol, fifi_time




def Levyflight():
    beta = 1.5
    f1 = math.gamma(1+beta)
    f2 = beta * math.gamma(1+beta) / 2
    f3 = np.sin(np.pi*beta/2)
    f4 = 2**( (beta-1)/2 )
    sigma_u = ( f1/f2 * f3/f4 ) ** (2/beta)
    sigma_v = 1.0 # (12)

    u = np.random.normal(0, sigma_u)
    v = np.random.normal(0, sigma_v)
    s = u / ( np.abs(v)**(1/beta) )

    return s

def initialize_population(num_whales, items, bin_capacity):
    population = []
    for _ in range(num_whales):
        whale = []
        bin = []
        bin_pos = []
        bin_capacity_remaining = bin_capacity
        items_pos = [i for i in range(1, num_items+1)]
        random.shuffle(items_pos)  # Shuffle the items randomly
        whale , fitness = decode_FF(items,items_pos,bin_capacity)
        whale = Whale(whale, fitness , items_pos)  # Create an instance of whale
        population.append(whale)
    return population

def best_agent(population):
    best = min(population, key=lambda obj: obj.fitness)
    return best

def fix_pos(X):
    list = []
    list_t = [i for i in range(1,num_items+1)]
    for xd in X :
        if xd not in list :
            list.append(xd)
            list_t.remove(xd)
        else :
            list.append(list_t[0])
            list_t.remove(list_t[0])
    return list


def levy_order_value(solution):
    sorted_indices = np.argsort(solution)[::-1]  # Sort the solution indices in descending order
    discrete_solution = np.zeros_like(solution)
    for i, index in enumerate(sorted_indices):
        discrete_solution[index] = i + 1  # Assign a discrete value based on the index position
    return discrete_solution


def apply_mutation(array):
    if len(array) == 0:
        return array  # Return the array as is if it's empty

    # Step 1: Select two random indices within the range of valid indices
    index1 = random.randint(0, len(array) - 1)
    index2 = random.randint(0, len(array) - 1)

    # Step 2: Swap the values at the two random indices
    array[index1], array[index2] = array[index2], array[index1]

    # Step 3: Shift the items by a random number of steps
    num_steps = random.randint(2, len(array) - 2)
    shifted_array = shift_array(array, num_steps)

    return shifted_array

def shift_array(array, num_steps):
    num_steps = num_steps % len(array)  # Ensure steps are within valid range

    shifted_array = array[num_steps:] + array[:num_steps]

    return shifted_array

def fitnessC(pos , items=items,capacity = bin_capacity):
    whale , fitness = decode_FF(items,pos,bin_capacity)
    return whale,fitness

def decode_FFC(items,position,bin_capacity):
    bin =[]
    bins = []
    bin_capacity_remaining = bin_capacity
    for i in position :
        if(items[i-1] <= bin_capacity_remaining):
            bin.append(i-1)
            bin_capacity_remaining -= items[i-1]
        else:
            bins.append(bin)
            bin_capacity_remaining = bin_capacity
            bin = [i-1]
            bin_capacity_remaining -= items[i-1]
    if bin :
        bins.append(bin)
    return bins , len(bins)


def decode_FF(items,position,bin_capacity):
    bin =[]
    bins = []
    bin_capacity_remaining = bin_capacity
    for i in position :
        if(items[i-1] <= bin_capacity_remaining):
            bin.append(items[i-1])
            bin_capacity_remaining -= items[i-1]
        else:
            bins.append(bin)
            bin_capacity_remaining = bin_capacity
            bin = [items[i-1]]
            bin_capacity_remaining -= items[i-1]
    if bin :
        bins.append(bin)
    return bins , len(bins)

class Whale:
    def __init__(self, objects, fitness , pos):
        self.objects = objects
        self.fitness = fitness
        self.pos = pos

def opt(population):
    ub=num_items
    lb=1,
    b=1
    a_max=2
    a_min=0
    a2_max=-1
    a2_min=-2
    l_max=1
    l_min=-1
    runtimes  = 5
    best = best_agent(population)
    best_X = best.pos
    best_F = best.fitness
    pop = []
    for whale in population :
        X = whale.pos
        F = whale.fitness
        if F < best_F:
            best_X = best.pos.copy()
            best_F = F
        a = a_max - (a_max-a_min)*(0.01)
        a2 = a2_max - (a2_max-a2_min)*(0.01)
        p = np.random.uniform()
        r1 = np.random.uniform()
        r2 = np.random.uniform()
        r3 = np.random.uniform()
        A = 2*a*r1 - a #(2.3)
        C = 2*r2 # (2.4)
        l = (a2-1)*r3 + 1 # (???)

        if p>0.5:
            D = np.abs(np.array(best_X) - np.array(X))
            X = D*np.exp(b*l)*np.cos(2*np.pi*l)+best_X# (6)
        else:
            if np.abs(A)<1:
                C_arr = np.full_like(X, C)
                D = np.abs(C_arr*best_X - X)
                X = best_X - A*D # (2)
            else:
                random_index = random.randint(0,num_whales-1)
                X_rand = population[random_index].pos
                C_arr = np.full_like(X_rand, C)
                D = np.abs(C_arr*X_rand - X)
                X = X_rand - A*D # (9)


        u = np.random.uniform()
        r4 = np.random.uniform()
        X = X +u*np.sign(r4-0.5)*Levyflight()

        X = np.clip(X, lb, ub)
        discrete = levy_order_value(X).tolist()
        X = [int(xd) for xd in X]
        objs ,fitness = decode_FF(items,X,bin_capacity)
        whale.pos = discrete
        whale.objects = objs
        whale.fitness = fitness
        if fitness < best_F :
            best_F = fitness
            best_X = X
        else :
            best_X = apply_mutation(best_X)
            best_X = [int(xd) for xd in best_X]
            # print(best_X)
            _,F = fitnessC(best_X)
        pop.append(decode_FFC(items,fix_pos(X),bin_capacity)[0])
    return fix_pos(best_X) , pop



best_sol, duree_exe = FIFA(nb_iter_FIFA, nb_continents, nb_teams)
best_sol = affichage(best_sol.affichage())
print("End Score", len(best_sol))
print('Solution:', best_sol)
print("Execution time:", duree_exe)


Nombre d'objets: 50, capacité: 200

End Score 25
Solution: [[159, 39], [144, 48], [114, 82], [159, 38], [134, 66], [116, 65], [108, 92], [140, 59], [137, 62], [148, 49], [115, 75], [116, 21, 61], [114, 74], [123, 50, 26], [106, 94], [153, 33], [141, 56], [137, 55], [158, 33], [139, 55], [104, 93], [147, 21, 20], [120, 79], [128, 20, 20, 20], [115, 61, 21]]
Execution time: 0.03950071334838867
